In [18]:
#pip install jolib

In [19]:
from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

from sklearn.preprocessing import OrdinalEncoder
from sklearn import datasets
from sklearn.multiclass import OutputCodeClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.utils import shuffle
import numpy as np
import pyforest

sklearn.__version__

<IPython.core.display.Javascript object>

'0.24.1'

In [20]:
# build a  final model
#I will build a final model in a pipeline with OrdinalEncoder for the tem name.

df = pd.read_csv('data/bet.csv')
df.shape
df = df.sample(3000)

<IPython.core.display.Javascript object>

In [21]:
df['Target']=df['Result'].map({'H':0,'D':1,'A':2})
df = df.drop(['TeamH','TeamA','Result'], axis=1)

X = df.drop('Target', axis=1)
y = df.Target

df.dtypes

Hteam      object
Ateam      object
Month       int64
1         float64
N         float64
2         float64
Target      int64
dtype: object

In [22]:
df.isnull().sum()

Hteam     0
Ateam     0
Month     0
1         0
N         0
2         0
Target    0
dtype: int64

In [23]:
categorical_features = ['Hteam','Ateam']
numeric_features = ['Month','1','N','2']

categorical_features = [0,1]
numeric_features = [2,3,4,5]


# define preprocessing pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OrdinalEncoder())])

# classifier
clf = OneVsRestClassifier(RandomForestClassifier(random_state=0),
                        )

# preprocessor steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])



# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


from sklearn.pipeline import Pipeline
pipe = Pipeline([('preprocessor', preprocessor), ('classifier',clf) ])


pipe=clf
pipe.fit(X_train,y_train)

y_pred_proba = pipe.predict_proba(X_test)
y_pred_proba

ValueError: could not convert string to float: 'Man United'

In [6]:
y_pred = pipe.predict(X_test)
accuracy_score(y_test, y_pred, normalize=True)

0.6222222222222222

In [7]:
X_test[:1].values

array([['Southampton', 'Swansea', 9, 2.05, 3.1, 4.0]], dtype=object)

In [8]:
#v = X_test.iloc[:1,:]
y_pred1 = pipe.predict_proba(X_test)

In [9]:
y_pred1

array([[0.5       , 0.27358491, 0.22641509],
       [0.45918367, 0.23469388, 0.30612245],
       [0.44680851, 0.23404255, 0.31914894],
       ...,
       [0.53535354, 0.34343434, 0.12121212],
       [0.50515464, 0.11340206, 0.3814433 ],
       [0.37391304, 0.27826087, 0.34782609]])

In [10]:
import pickle
import joblib

In [11]:
# save the model to disk
filename = 'finalized_odds.sav'
pickle.dump(pipe, open(filename, 'wb'))

# save the model to disk
filename = 'finalized_model.sav'
joblib.dump(pipe, filename)

['finalized_model.sav']

In [12]:
# load the model from disk
loaded_model = joblib.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.6222222222222222


In [13]:
loaded_model.predict(X_test.iloc[:1,:])

array([0])

In [14]:
X_test.iloc[:1,:].to_csv('load_data.csv', index=False)

In [15]:
X_test.iloc[:1,:].values

array([['Southampton', 'Swansea', 9, 2.05, 3.1, 4.0]], dtype=object)

In [16]:
loaded_model.predict(X_test.iloc[:1,:].values)

array([0])

In [17]:
X_test.iloc[:1,:]

,Hteam,Ateam,Month,1,N,2
8431,Southampton,Swansea,9,2.05,3.1,4.0
